In [1]:
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
import string
import MeCab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import unicodedata
from gensim import corpora
from gensim import models
from pprint import pprint
import time
import math

In [2]:
def YahooNewsAnalytics(n=30):
    i = 0
    url = "https://news.yahoo.co.jp/topics/top-picks"
    URL = "https://news.yahoo.co.jp/"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    all_page_links = []
    all_page_links.append(url)
    while True:
        try:
            next = soup.find("li", class_="pagination_item-next").find("a")["href"]
            next_link = URL + next
            all_page_links.append(next_link)
            next_res = requests.get(next_link)
            soup = BeautifulSoup(next_res.text, "html.parser")
        except:
            break
    
    all_links = []
    for url in all_page_links: # all_page_links: 全てのニュースのリスト
        print(url, end='\n\n')
        res = requests.get(url) # url: 25個分のニュースのリスト
        soup = BeautifulSoup(res.text, "html.parser")
        page_soup = soup.find_all("a", class_="newsFeed_item_link")
        for href in page_soup:
            news = []
            link = href["href"] # link: 一つのニュースのリンク(本文は一部のみ)
            link_res = requests.get(link)
            href_soup = BeautifulSoup(link_res.text, "html.parser")
            title_link = href_soup.find("a", class_="sc-eAyhxF")["href"] # title_link: 本文
            news.append(title_link)
            res1 = requests.get(title_link)
            soup1 = BeautifulSoup(res1.text, "html.parser")
            while True:
                try:
                    next_link = soup1.find("li", class_="pagination_item-next").find("a")["href"]
                    next_link = URL + next_link
                    next_res = requests.get(next_link)
                    next_soup = BeautifulSoup(next_res.text, "html.parser")
                    news.append(next_soup)
                except Exception:
                    all_links.append(news)
                    i += 1
                    print('\r{0}記事'.format(i), end='')
                    if i >= n:
                        return all_links
                    break
    return all_links

In [4]:
YahooNewsAnalytics(10)

https://news.yahoo.co.jp/topics/top-picks

10記事

[['https://headlines.yahoo.co.jp/article?a=20201102-00000004-friday-base'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-00202469-fnn-soci'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-02362953-tospoweb-fight'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-00000007-jij_afp-int'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-21011444-nksports-base'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-00000050-kyodonews-soci'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-00010000-chibatopi-l12'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-00000515-sanspo-ent'],
 ['https://headlines.yahoo.co.jp/hl?a=20201102-00000004-hbcv-hok'],
 ['https://headlines.yahoo.co.jp/hl?a=20201101-00000109-mai-soci']]